In [ ]:
import pandas as pd
import numpy as np

if __name__ == "__main__":
    movie=pd.read_csv('ml-latest/movies.csv')
    rating=pd.read_csv('ml-latest/ratings.csv')

In [ ]:
#그대로 user-movie matrix 를 만들기엔 너무 큼
#movie_pivot = rating.pivot_table(columns='userid',index='movieId',values='rating')

# 500개 이상 평가한 user만 filtering
x = rating['userId'].value_counts()>500
print(len(x[x]))
thin_rating = rating[rating['userId'].isin(x[x].index)]
movie_pivot = thin_rating.pivot_table(columns='userId',index='movieId',values='rating')

# movie도 rating수로 filtering
y = thin_rating['movieId'].value_counts()>50
print(len(y[y]))
thin_rating= thin_rating[thin_rating['movieId'].isin(y[y].index)]
movie_pivot = thin_rating.pivot_rable(columns='userId',index='movieId',values='rating')

#nan은 일단 0으로 채우기
movie_pivot.fillna(0,inplace=True)




## user24가 Jumanji (movieId = 2) 를 몇점으로 평가할까?

In [ ]:
# Item-Item Collaborative Filtering

from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

movie_sparse = csr_matrix(movie_pivot)
model = NearestNeighbors(n_neighbors=10, algorithm='brute',metric ='cosine')
model.fit(movie_sparse)


In [ ]:
movie_pivot.iloc[1,:].value.reshape(1,-1)

#이 item과 비슷하게 user들에게 평가받은 item을 nearest neighbor로 찾음
distance,suggestions = model.kneighbors(movie_pivot.iloc[1,:].values.reshape(1,-1))
distance,suggestions


In [ ]:
movie[movie['movieId'].isin(movie_pivot.index[suggestions[0]])]

#비슷한 top 10 movies 의 user 들의 평가
movie_pivot.iloc[suggestions[0]]


### cosine distance 의 weighted average

In [ ]:
cosineSim = 1 - distance[0,1:]
print(cosineSim)


In [ ]:
weight_sum = cosineSim.sum()
weight_sum

In [ ]:
neighbor = movie_pivot.iloc[suggestions[0, 1:]]
neighbor

In [ ]:
weighted_est = [0] * len(neighbor.columns)
for i in range(len(neighbor.columns)):
    col_idx = neighbor.columns[i]
    sum = 0
    for j in range(len(neighbor)):
        sum += cosineSim[j] * neighbor[col_idx].iloc[j]
    weighted_est[i] = sum / weight_sum

In [ ]:
movie_pivot_est = movie_pivot.copy()
movie_pivot_est.loc[2] = weighted_est
movie_pivot_est.loc[2]

In [1]:
movie_pivot_est[24][2]

NameError: name 'movie_pivot_est' is not defined

### pearson correlation으로 바꾸기

In [ ]:
model = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='correlation')
model.fit(movie_pivot)

In [ ]:
distance,suggestions = model.kneighbors(movie_pivot.iloc[1,:].values.reshape(1,-1))
distance,suggestions

In [ ]:
movie_corr = movie_pivot.iloc[suggestions[0, 0:]]
movie_corr[:]

In [ ]:
weight_corr = np.corrcoef(movie_corr)[0]
print(weight_corr)

In [ ]:
# 분모 구해놓기
weight_sum = weight_corr[1:].sum()
weight_sum

In [ ]:
weighted_est = [0] * len(movie_corr[1:].columns)
for i in range(len(movie_corr[1:].columns)):
    col_idx = movie_corr[1:].columns[i]
    sum = 0
    for j in range(len(movie_corr[1:])):
        sum += weight_corr[1:][j] * movie_corr[1:][col_idx].iloc[j]
    weighted_est[i] = sum / weight_sum

In [ ]:
movie_pivot_est = movie_pivot.copy()
movie_pivot_est.loc[2] = weighted_est
movie_pivot_est.head()

In [ ]:
movie_pivot_est[24][2]

### user-user CF

In [ ]:
user_pivot = movie_pivot.T

In [ ]:
model = NearestNeighbors(n_neighbors=10, algorithm='brute', metric='correlation')
model.fit(user_pivot)

In [ ]:
distance,suggestions = model.kneighbors(\
    user_pivot.iloc[1,:].values.reshape(1,-1))

print(suggestions)

In [ ]:
user_corr = user_pivot.iloc[suggestions[0]]
user_corr[:]

In [ ]:
# 상관 계수 구하기

weight_corr = np.corrcoef(user_corr)[0]
weight_corr

In [ ]:
# 미리 더 해놓기
weight_corr_sum = weight_corr[1:].sum()
weight_corr_sum

In [ ]:
weighted_est = [0] * len(user_corr[1:].columns)
for i in range(len(user_corr[1:].columns)):
    col_idx = user_corr[1:].columns[i]
    sum = 0
    for j in range(len(user_corr[1:])):
        sum += weight_corr[1:][j] * user_corr[1:][col_idx].iloc[j]
    weighted_est[i] = sum / weight_corr_sum

In [ ]:
user_pivot.loc[24] = weighted_est
user_pivot.head()
user_pivot[2][24]